<a href="https://colab.research.google.com/github/alexandremendesb12/yolo-train/blob/main/YoloStreets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
%mkdir develop
%cd develop

In [24]:
!git clone https://github.com/alexandremendesb12/yolo-train.git
%cd yolo-train/

In [24]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git
%cd OIDv4_ToolKit/

In [ ]:
!python3 main.py downloader --classes Ambulance Bus Car Motorcycle Truck Person --limit 1000 --type_csv train

In [ ]:
!python3 main.py downloader --classes Ambulance Bus Car Motorcycle Truck Person --limit 200 --type_csv  validation

In [24]:
%cd ..
!git clone https://github.com/ultralytics/yolov5.git
%cd OIDv4_ToolKit/

In [ ]:
!git clone https://gist.github.com/b6e5d60343c88f358742aa9e0de2cc3f.git

In [ ]:
!python3 b6e5d60343c88f358742aa9e0de2cc3f/oid_to_yolo.py --voc_dir /develop/yolo-train/dataset/test --yolo_dir /develop/yolo-train/dataset/train

In [ ]:
%cd ..
%cd yolov5/

In [22]:
!python3 train.py --img 640 --batch 48 --epochs 10 --data ../dataset/data.yaml --weights yolov5s.pt --cache

In [24]:
import cv2
import torch

def draw_boxes(img, results):
    for *box, conf, cls in results.xyxy[0].cpu().numpy():
        x1, y1, x2, y2 = map(int, box)
        label = f'{results.names[int(cls)]} {conf:.2f}'
        img = cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        img = cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return img

def load_model(weights_path):
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
    model.eval()
    return model

def run_inference_on_video(model, video_path, output_path):

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model(img_rgb)

        img_result = draw_boxes(frame, results)

        out.write(img_result)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":

    weights_path = '/develop/yolo-train/yolov5/runs/train/exp/weights/best.pt'
    video_path = '../videos/origin.mp4'
    output_path = '../videos/output_video.mp4'

    model = load_model(weights_path)

    run_inference_on_video(model, video_path, output_path)
